In [1]:
from transformers import pipeline

model_name_or_path=r"E:\code\chinese-roberta-wwm-ext"

In [21]:
classifier = pipeline("zero-shot-classification", model = model_name_or_path, device=0)
classifier(
    "感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】",
    candidate_labels=["垃圾短信", "正常短信"],
    hypothesis_template = "这条信息是{}。"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at E:\code\chinese-roberta-wwm-ext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': '感谢致电杭州萧山全金釜韩国烧烤店，本店位于金城路xxx号。韩式烧烤等，价格实惠、欢迎惠顾【全金釜韩国烧烤店】',
 'labels': ['正常短信', '垃圾短信'],
 'scores': [0.5054343938827515, 0.4945656359195709]}

In [4]:
import pandas as pd
import numpy as np

In [5]:
#1、加载数据
train_df = pd.read_csv(r'data.csv', encoding='utf-8', header=None, names=['label','review'])
print(train_df.shape)

sentences = list(train_df['review'][1:])
label =train_df['label'][1:].values

(1243, 2)


In [6]:
labels = []
for s in label:
    labels.append(int(s))

In [8]:
classifier = pipeline("zero-shot-classification", model = model_name_or_path, device=0, batch_size=16)

results = classifier(
    sentences[:300],
    candidate_labels=["垃圾短信", "正常短信"],
    hypothesis_template = "这条信息是{}。",
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at E:\code\chinese-roberta-wwm-ext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [9]:
pre = []
for each in results:
    if each['labels'][0] == '正常短信':
        pre.append(0)
    else:
        pre.append(1)

In [10]:
from sklearn import metrics
classify_report = metrics.classification_report(pre, labels[:300], digits = 4) #分类报告 support测试集样本数
print(classify_report) 
confusion_matrix = metrics.confusion_matrix(pre, labels[:300]) #混淆矩阵
print(confusion_matrix) 

              precision    recall  f1-score   support

           0     0.5301    0.8598    0.6558       164
           1     0.3235    0.0809    0.1294       136

    accuracy                         0.5067       300
   macro avg     0.4268    0.4703    0.3926       300
weighted avg     0.4364    0.5067    0.4172       300

[[141  23]
 [125  11]]


In [26]:
classifier(
    "2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。",
    candidate_labels=["盗窃", "抢劫", "走私、贩卖、运输、制造毒品", "危险驾驶"],
    hypothesis_template = "这条陈述的罪名是{}。"
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at E:\code\chinese-roberta-wwm-ext and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': '2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。',
 'labels': ['盗窃', '危险驾驶', '抢劫', '走私、贩卖、运输、制造毒品'],
 'scores': [0.25567951798439026,
  0.2530823349952698,
  0.2519243657588959,
  0.2393137663602829]}

In [22]:
classifier(
    "你好！我有脚气十几年了，手也脱皮，是不是真菌感染，可以吃伊曲康唑吗？",
    candidate_labels=["小儿内科", "妇科", "皮肤科", "呼吸内科", "内分泌科", "耳鼻喉科"]
)

{'sequence': '你好！我有脚气十几年了，手也脱皮，是不是真菌感染，可以吃伊曲康唑吗？',
 'labels': ['皮肤科', '呼吸内科', '耳鼻喉科', '妇科', '小儿内科', '内分泌科'],
 'scores': [0.16989482939243317,
  0.16912688314914703,
  0.16610512137413025,
  0.1659039705991745,
  0.16448993980884552,
  0.16447928547859192]}

In [11]:
fill_mask = pipeline("fill-mask", model=model_name_or_path)
result = fill_mask("这条陈述的罪名是危险驾驶吗？答案是[MASK]。2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。")

print(result)
print(result[0]['score'])

Some weights of the model checkpoint at E:\code\chinese-roberta-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.8795590400695801, 'token': 3221, 'token_str': '是', 'sequence': '这 条 陈 述 的 罪 名 是 危 险 驾 驶 吗 ？ 答 案 是 是 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。'}, {'score': 0.05487998574972153, 'token': 4638, 'token_str': '的', 'sequence': '这 条 陈 述 的 罪 名 是 危 险 驾 驶 吗 ？ 答 案 是 的 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。'}, {'score': 0.016319895163178444, 'token': 679, 'token_str': '不', 'sequence': '这 条 陈 述 的 罪 名 是 危 险 驾 驶 吗 ？ 答 案 是 不 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。'}, {'score': 0.008570448495447636, 'token': 2190, 'token_str': '对', 'sequence': '这 条 陈 述 的 罪 名 是 危 险 驾 驶 吗 ？ 答 案 是 对 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。'}, {'score': 0.0064257062040269375, 'token': 7231, 'token_str': '错', 'sequence': '这 条 陈 述 的 罪 名 是 危 险 驾 驶 吗 ？ 

In [12]:
from transformers import pipeline

model_name_or_path=r"E:\code\chinese-roberta-wwm-ext"

fill_mask = pipeline("fill-mask", model=model_name_or_path)
result = fill_mask("这条陈述的罪名是[MASK][MASK]。2010年12月21日10时许，被告人王某趁武强县金音建筑工地工友张某不在宿舍之际，盗窃张某枕头下现金3500元。")

for mask in result:
    for sequence in mask:
        print(sequence)

Some weights of the model checkpoint at E:\code\chinese-roberta-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'score': 0.9837144017219543, 'token': 4668, 'token_str': '盗', 'sequence': '[CLS] 这 条 陈 述 的 罪 名 是 盗 [MASK] 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。 [SEP]'}
{'score': 0.005338470917195082, 'token': 2843, 'token_str': '抢', 'sequence': '[CLS] 这 条 陈 述 的 罪 名 是 抢 [MASK] 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。 [SEP]'}
{'score': 0.0021143672056496143, 'token': 4961, 'token_str': '窃', 'sequence': '[CLS] 这 条 陈 述 的 罪 名 是 窃 [MASK] 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。 [SEP]'}
{'score': 0.0015745058190077543, 'token': 4306, 'token_str': '犯', 'sequence': '[CLS] 这 条 陈 述 的 罪 名 是 犯 [MASK] 。 2010 年 12 月 21 日 10 时 许 ， 被 告 人 王 某 趁 武 强 县 金 音 建 筑 工 地 工 友 张 某 不 在 宿 舍 之 际 ， 盗 窃 张 某 枕 头 下 现 金 3500 元 。 [SEP]'}
{'score': 0.001478593796491623, 'token': 6400, 'token_str': '诈', 'sequence': '[CLS] 这 条 陈 述 的 罪 名 是 诈 [M